In [1]:
from deploy.domain import model
from deploy.adapters import orm
from deploy.bootstrap import bootstrap

from sqlalchemy import select

In [2]:
bus = bootstrap()

In [3]:
with bus.uow as uow:
    [user_in_db] = uow.users.get("jochen")
    user = model.User(**user_in_db.dict())

In [4]:
user

User(id=2, name=jochen)